<a href="https://colab.research.google.com/github/DakshLadha/Alarm-Clock/blob/main/Feature-Extraction(MobileNet-DenseNet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from collections import Counter
import cv2
import time
import glob
import skimage
import numpy as np
import pandas as pd
import seaborn as sn
from tqdm import tqdm
from os import listdir
import matplotlib.pyplot as plt
from skimage.transform import resize
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

# !pip install FS
from sklearn.neighbors import KNeighborsClassifier
#from drive.MyDrive.FS.woa import jfs   # change this to switch algorithm

sn.set()

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from time import time

from sklearn import metrics
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score,f1_score,recall_score,cohen_kappa_score,precision_score
from sklearn.utils import compute_class_weight
from sklearn.preprocessing import MinMaxScaler,LabelBinarizer, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier # KNeighborsClassifier
from sklearn.svm import SVC # SVM
from sklearn.ensemble import RandomForestClassifier # RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier # AdaBoostClassifier
from xgboost import XGBClassifier # XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16 # VGG16
from tensorflow.keras.applications.vgg19 import VGG19 # VGG19
from tensorflow.keras.applications.resnet50 import ResNet50 # ResNet50
#from tensorflow.keras.applications.resnet9 import ResNet9 # ResNet50
from tensorflow.keras.applications import ResNet101,InceptionResNetV2 # ResNet 101
from tensorflow.keras.applications.xception import Xception # Xception
from tensorflow.keras.applications.mobilenet import MobileNet # MobileNet
from tensorflow.keras.applications.densenet import DenseNet169 # DenseNet169
from tensorflow.keras.applications.densenet import DenseNet121 # DenseNet121
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 # MobileNetV2
from tensorflow.keras.applications.inception_v3 import InceptionV3 # InceptionV3
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D,Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
def loadImages(path, limit=100):
    sample = []
    count = 0

    for filename in glob.glob(path):
        if limit is not None and count >= limit:
            break

        img = cv2.imread(filename)
        img = skimage.transform.resize(img, (224, 224, 3))
        IMG = np.array(img)
        sample.append(IMG)

        count += 1

    return sample

In [4]:
train_path1 = '/content/drive/MyDrive/Peach___Bacterial_spot/*.JPG'
train_path2 = '/content/drive/MyDrive/Peach___healthy/*.JPG'
train_path3 = '/content/drive/MyDrive/Strawberry___Leaf_scorch/*.JPG'
train_path4 = '/content/drive/MyDrive/Strawberry___healthy/*.JPG'
#train_path5 = 'drive/My Drive/train/tlb/*.JPG'
#train_path6 = 'drive/My Drive/train/tlm/*.JPG'
#train_path7 = 'drive/My Drive/train/ttylcv/*.JPG'
#train_path8 = 'drive/My Drive/train/tts/*.JPG'
#train_path9 = 'drive/My Drive/train/ttmv/*.JPG'
#train_path10 = 'drive/My Drive/train/tslf/*.JPG'


train_1 = loadImages(train_path1)
train_2 = loadImages(train_path2)
train_3 = loadImages(train_path3)
train_4 = loadImages(train_path4)
#train_5 = loadImages(train_path5)
#train_6 = loadImages(train_path6)
#train_7 = loadImages(train_path7)
#train_8 = loadImages(train_path8)
#train_9 = loadImages(train_path9)
#train_10 = loadImages(train_path10)

#%% CREATION OF DATASETS

In [5]:
df_train_1 = pd.DataFrame({'image':train_1, 'label': '1'})
df_train_2 = pd.DataFrame({'image':train_2, 'label': '2'})
df_train_3 = pd.DataFrame({'image':train_3, 'label': '3'})
df_train_4 = pd.DataFrame({'image':train_4, 'label': '4'})
#df_train_5 = pd.DataFrame({'image':train_5, 'label': '5'})
#df_train_6 = pd.DataFrame({'image':train_6, 'label': '6'})
#df_train_7 = pd.DataFrame({'image':train_7, 'label': '7'})
#df_train_8 = pd.DataFrame({'image':train_8, 'label': '8'})
#df_train_9 = pd.DataFrame({'image':train_9, 'label': '9'})
#df_train_10 = pd.DataFrame({'image':train_10, 'label': '10'})

final_data = [df_train_1,df_train_2, df_train_3, df_train_4]
final_data = pd.concat(final_data)

print("Entire data size:",final_data.shape)

Entire data size: (400, 2)


In [6]:
#%% TRAIN LABEL SEPARATION

train_data = final_data['image']
labels = final_data['label']

print("Labels Count:",Counter(np.array(labels)))

onehot = LabelEncoder()
labels = onehot.fit_transform(labels)
print(labels.size)

X_train, X_test, y_train, y_test = train_test_split(train_data, labels,
                                                  test_size = 0.3,
                                                  stratify = labels,
                                                  )

X_train = np.array(X_train)
X_test = np.array(X_test)

print(y_train.size)

Labels Count: Counter({'1': 100, '2': 100, '3': 100, '4': 100})
400
280


In [7]:
print(train_data.shape)
print(np.unique(y_train))

(400,)
[0 1 2 3]


In [8]:
x_train = np.empty((len(X_train),X_train[0].shape[0],X_train[0].shape[1],X_train[0].shape[2]))
for i,x in enumerate(X_train):
    x_train[i]=X_train[i]
print(x_train.shape)
x_test= np.empty((len(X_test),X_test[0].shape[0],X_test[0].shape[1],X_test[0].shape[2]))
for i,x in enumerate(X_test):
    x_test[i]=X_test[i]
print(x_test.shape)

(280, 224, 224, 3)
(120, 224, 224, 3)


In [9]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224,224,3))
x = base_model.output
x =  Dropout(0.5)(x)
x = Flatten()(x)

model_feat = Model(inputs=base_model.input,outputs=x)

mobile_train_features = model_feat.predict(x_train)
mobile_test_features=model_feat.predict(x_test)
print(mobile_train_features)
print(mobile_test_features)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step
[[0.        2.8985481 0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [0.        3.0746684 0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        5.129878  0.        ... 0.        0.        0.       ]]
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         3.1745105  0.         ... 0.         0.         0.        ]
 [0.         0.98361254 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.61998844 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.04434633]
 [0.         0.90555525 0.         ... 0.         0.         0.        ]]


In [10]:
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(224,224,3))
y = base_model.output
y =  Dropout(0.5)(y)
y = Flatten()(y)

model_feat = Model(inputs=base_model.input,outputs=y)

dense_train_features = model_feat.predict(x_train)
dense_test_features=model_feat.predict(x_test)
print(dense_train_features)
print(dense_test_features)

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 64s 7s/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 6s/step
[[0.0000000e+00 4.4753030e-04 0.0000000e+00 ... 6.5886045e-01
  0.0000000e+00 2.8094332e+00]
 [5.2728155e-04 1.0579238e-03 0.0000000e+00 ... 3.1045461e+00
  8.7979108e-02 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 7.1028774e-03 ... 0.0000000e+00
  0.0000000e+00 1.0765806e-01]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 2.6031337e+00
  7.4842721e-02 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 4.9169175e-05 ... 1.9374096e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 4.2393757e-04 0.0000000e+00 ... 2.9219704e+00
  0.0000000e+00 0.0000000e+00]]
[[0.0000000e+00 0.0000000e+00 1.7534562e-03 ... 2.4203317e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.6948424e-03 ... 8.4434261e+00
  0.0000000e+00 5.2924490e-01]
 [1.2036539e-03 2.2797046e-02 0.0000000e+00 ... 2.3833659e+00
  7.2246641e-02 1.8010060e+00]
 ...
 [0.0000000e+00 0.0000000e+

In [11]:
csv_mobile = pd.DataFrame(mobile_train_features)
csv_densenet = pd.DataFrame(dense_train_features)
print(np.unique(y_train))
csv_label = pd.DataFrame(y_train)
final_csv = [csv_mobile,csv_densenet,csv_label]
#final_csv = pd.concat(final_csv,axis=1)
#final_csv.to_csv('train_features_2.csv')
new = pd.concat(final_csv, axis=1, join='inner')
new.to_csv('train_features_2.csv')
!cp train_features_2.csv "drive/My Drive/"

[0 1 2 3]


In [12]:
train_features = np.append(mobile_train_features,dense_train_features,axis=1)
test_features = np.append(mobile_test_features,dense_test_features,axis=1)

In [13]:
x_train_acc, x_val_acc, y_train_acc, y_val_acc = train_test_split(train_features,y_train,
                                                  test_size = 0.2,
                                                  stratify = y_train,
                                                  shuffle = True,
                                                  random_state = 42)
X_test,y_test=test_features,y_test

print('length X_train:', len(x_train_acc))
print('length y_train:', len(y_train_acc))

print('length X_val:',  len(x_val_acc))
print('length y_val:', len(y_val_acc))

print('length X_test:',  len(X_test))
print('length y_test:', len(y_test))

length X_train: 224
length y_train: 224
length X_val: 56
length y_val: 56
length X_test: 120
length y_test: 120


In [14]:
def classifier_summary(pipeline, X_train, y_train, X_val, y_val,X_test,y_test):

    start = time()
    sentiment_fit = pipeline.fit(X_train, y_train)
    end = time()

    y_pred_train= sentiment_fit.predict(X_train)
    y_pred_val = sentiment_fit.predict(X_val)
    y_pred_test = sentiment_fit.predict(X_test)

    train_accuracy = np.round(accuracy_score(y_train, y_pred_train),4)*100
    train_precision = np.round(precision_score(y_train, y_pred_train, average='weighted'),4)
    train_recall = np.round(recall_score(y_train, y_pred_train, average='weighted'),4)
    train_F1 = np.round(f1_score(y_train, y_pred_train, average='weighted'),4)
    train_kappa =  np.round(cohen_kappa_score(y_train, y_pred_train),4)
    train_confusion_matrix = confusion_matrix(y_train,y_pred_train)

    val_accuracy = np.round(accuracy_score(y_val, y_pred_val),4)*100
    val_precision = np.round(precision_score(y_val, y_pred_val, average='weighted'),4)
    val_recall = np.round(recall_score(y_val, y_pred_val, average='weighted'),4)
    val_F1 = np.round(f1_score(y_val, y_pred_val, average='weighted'),4)
    val_kappa =  np.round(cohen_kappa_score(y_val, y_pred_val),4)
    val_confusion_matrix = confusion_matrix(y_val,y_pred_val)

    test_accuracy = np.round(accuracy_score(y_test, y_pred_test),4)*100
    test_precision = np.round(precision_score(y_test, y_pred_test, average='weighted'),4)
    test_recall = np.round(recall_score(y_test, y_pred_test, average='weighted'),4)
    test_F1 = np.round(f1_score(y_test, y_pred_test, average='weighted'),4)
    test_kappa =  np.round(cohen_kappa_score(y_test, y_pred_test),4)
    test_confusion_matrix = confusion_matrix(y_test,y_pred_test)


    print()
    print("Time taken : {}".format(end-start))
    print('------------------------ Train Set Metrics------------------------')
    print()
    print("accuracy : {}%".format(train_accuracy))
    print("F1_score : {}".format(train_F1))
    print("Cohen Kappa Score : {} ".format(train_kappa))
    print("Recall : {}".format(train_recall))
    print("Precision : {}".format(train_precision))
    print("Confusion Matrix :\n {}".format(train_confusion_matrix))

    print()
    print('------------------------ Validation Set Metrics------------------------')
    print()
    print("accuracy : {}%".format(val_accuracy))
    print("F1_score : {}".format(val_F1))
    print("Cohen Kappa Score : {} ".format(val_kappa))
    print("Recall : {}".format(val_recall))
    print("Precision : {}".format(val_precision))
    print("Confusion Matrix :\n {}".format(val_confusion_matrix))

    print()
    print('------------------------ Test Set Metrics------------------------')
    print()
    print("accuracy : {}%".format(test_accuracy))
    print("F1_score : {}".format(test_F1))
    print("Cohen Kappa Score : {} ".format(test_kappa))
    print("Recall : {}".format(test_recall))
    print("Precision : {}".format(test_precision))
    print("Confusion Matrix : {}".format(test_confusion_matrix))

    print("-"*80)
    print()

In [15]:
names = [
        'SVM',
        "Random Forest Classifier",
        "AdaBoost Classifier",
        "XGB Classifier",
         ]
classifiers = [
    SVC(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
        ]

zipped_clf = zip(names,classifiers)

In [16]:
def classifier_comparator(X_train,y_train,X_val,y_val,X_test,y_test,classifier=zipped_clf):
    result = []
    for n,c in classifier:
        checker_pipeline = Pipeline([
            ('classifier', c)
        ])
        print("Fitting {} on features ".format(n))
        #print(c)
        classifier_summary(checker_pipeline,X_train, y_train, X_val, y_val,X_test,y_test)

classifier_comparator(x_train_acc,y_train_acc,x_val_acc,y_val_acc,X_test,y_test,classifier=zipped_clf)

Fitting SVM on features 

Time taken : 14.411726474761963
------------------------ Train Set Metrics------------------------

accuracy : 100.0%
F1_score : 1.0
Cohen Kappa Score : 1.0 
Recall : 1.0
Precision : 1.0
Confusion Matrix :
 [[56  0  0  0]
 [ 0 56  0  0]
 [ 0  0 56  0]
 [ 0  0  0 56]]

------------------------ Validation Set Metrics------------------------

accuracy : 98.21%
F1_score : 0.9821
Cohen Kappa Score : 0.9762 
Recall : 0.9821
Precision : 0.9833
Confusion Matrix :
 [[13  1  0  0]
 [ 0 14  0  0]
 [ 0  0 14  0]
 [ 0  0  0 14]]

------------------------ Test Set Metrics------------------------

accuracy : 96.67%
F1_score : 0.9665
Cohen Kappa Score : 0.9556 
Recall : 0.9667
Precision : 0.9706
Confusion Matrix : [[26  4  0  0]
 [ 0 30  0  0]
 [ 0  0 30  0]
 [ 0  0  0 30]]
--------------------------------------------------------------------------------

Fitting Random Forest Classifier on features 

Time taken : 2.3463995456695557
------------------------ Train Set Metrics--